In [55]:
import torch
import numpy as np 
 
# =====================
# generate data
def gen_fake_data(hidden_weight, hidden_bias):
    def rand_5_numbers_from1to10():
        return np.random.permutation(10)[:5]

    x_train = torch.Tensor([rand_5_numbers_from1to10() for i in range(60)]) 
    x_test = torch.Tensor([rand_5_numbers_from1to10() for i in range(60)]) 
    
    hidden_weight = torch.Tensor(hidden_weight).unsqueeze(-1) 
    hidden_bias = torch.Tensor([hidden_bias]) 

    y_train = torch.mm(x_train, hidden_weight) + hidden_bias
    y_test = torch.mm(x_test, hidden_weight) + hidden_bias
    return x_train, y_train, x_test, y_test
     
# gen training data sample 
x_train, y_train, x_test, y_test = gen_fake_data(hidden_weight=[1,3,2,4,5], hidden_bias=1)
print(x_train[:3])
print(y_train[:3])
# =====================

tensor([[0., 4., 2., 9., 1.],
        [4., 9., 2., 7., 8.],
        [1., 8., 3., 6., 0.]])
tensor([[ 58.],
        [104.],
        [ 56.]])


In [56]:
from torch.utils.data import DataLoader

# dataloader
# dataset_example should support operator index_selection for create the data_loader object
train_loader = DataLoader([(x_train[i], y_train[i]) for i in range(x_train.shape[0])], batch_size=10, collate_fn=None, shuffle=True)
test_loader = DataLoader([(x_test[i], y_test[i]) for i in range(x_test.shape[0])], batch_size=50, collate_fn=None, shuffle=True)

In [57]:
# define a model 
class ModelLinear:
    def __init__(self, input_size, output_size):
        self.weight = torch.rand((input_size, output_size),requires_grad=True, device=torch.device('cuda:0'))
        self.bias = torch.rand(output_size, requires_grad=True, device=torch.device('cuda:0') )

    def __call__(self, input_matrix):
        return torch.mm(input_matrix, self.weight) + self.bias

    def get_parameters(self):
        return [self.bias, self.weight]


## Practice part 
- train a model to find hidden "weights and bias" from training data.
- **TODO**, practice with learning rate equal to 0.005, 0.02 0.01 to see the result how the model work 

In [62]:
# =============
# train a model to find hidden "weights and bias" from training data.
# TODO, practice with learning rate equal to 0.005, 0.02 0.01 to see the result how the model work 
# =============

# create a new model
model = ModelLinear(5,1)

# train
lr = 0.002

i=0
MAX_EPOCHS=5000
for epoch in range(MAX_EPOCHS):
    loss = 100000
    for batch in train_loader:

        x, y_gold = batch
  
        x = x.cuda()
        y_gold = y_gold.cuda()
        
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)


        # Compute and loss = average ((out_put - pred) ^ 2)
        total_sample_in_mini_batch = y_pred.shape[0] 
        loss = sum((y_pred - y_gold) * (y_pred - y_gold)) * 1/total_sample_in_mini_batch

        # perform a backward pass (backpropagation) => to get the gradient values in Tensor weights
        loss.backward()

        # Optimizer step(), this similar to the SGD algorithm 
        with torch.no_grad():
            for param in model.get_parameters():
                param.add_(-lr * param.grad)
                param.grad.fill_(0)
    
    if loss.item() < 0.001:
        break
    print('epoch/batch: ', epoch, i,' loss: ', loss.item())
    i+=1

print(model.weight,  model.bias) 

epoch/batch:  0 0  loss:  38.480247497558594
epoch/batch:  1 1  loss:  35.1999626159668
epoch/batch:  2 2  loss:  27.283655166625977
epoch/batch:  3 3  loss:  10.864322662353516
epoch/batch:  4 4  loss:  6.0600738525390625
epoch/batch:  5 5  loss:  3.509241819381714
epoch/batch:  6 6  loss:  3.6849076747894287
epoch/batch:  7 7  loss:  2.16418719291687
epoch/batch:  8 8  loss:  1.2757031917572021
epoch/batch:  9 9  loss:  0.42720356583595276
epoch/batch:  10 10  loss:  0.6071180105209351
epoch/batch:  11 11  loss:  0.2020016461610794
epoch/batch:  12 12  loss:  0.17425628006458282
epoch/batch:  13 13  loss:  0.09398400038480759
epoch/batch:  14 14  loss:  0.10839567333459854
epoch/batch:  15 15  loss:  0.11964117735624313
epoch/batch:  16 16  loss:  0.07217350602149963
epoch/batch:  17 17  loss:  0.03018859401345253
epoch/batch:  18 18  loss:  0.014126230962574482
epoch/batch:  19 19  loss:  0.015111957676708698
epoch/batch:  20 20  loss:  0.012493959628045559
epoch/batch:  21 21  loss